In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "lampe01" # username from the json file
os.environ['KAGGLE_KEY'] = "0fe21bb583c2cd3230e9d928c96e27f7" # key from the json file
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CirCor"

In [3]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install Ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip uninstall scikit-learn

Found existing installation: scikit-learn 1.1.1
Uninstalling scikit-learn-1.1.1:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/scikit_learn-1.1.1.dist-info/*
    /usr/local/lib/python3.8/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.8/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-1.1.1


In [7]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.1.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyaudioprocessing 1.1.13 requires scikit-learn==1.1.1; python_version >= "3.8", but you have scikit-learn 1.1.3 which is incompatible.


In [8]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
pip install pyAudioProcessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_learn-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3


In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import random
import itertools
import os
import sys
import pywt
import librosa
import sklearn
import scipy
import scipy.io
import scipy.io.wavfile
from scipy import signal
from scipy.stats import kurtosis, skew, tstd
from scipy.signal import periodogram, welch, butter, lfilter, freqz
from os import path
from pydub import AudioSegment
from collections import defaultdict
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Dense, Dropout, AlphaDropout, Activation, Flatten, LSTM
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping,ReduceLROnPlateau,ProgbarLogger
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, recall_score, classification_report, auc, roc_curve, precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from pyAudioProcessing.extract_features import get_features

In [11]:
mdata = pd.read_csv('/content/drive/MyDrive/ReCirCor/training_data.csv')
input_dir = '/content/drive/MyDrive/ReCirCor/training_data/training_data'

In [12]:
#Gets all the txt files
def getfilenames(inputdir):
    Fname = list()
    Fname = sorted([fname for fname in os.listdir(inputdir) if not fname.startswith('.')and fname.endswith('.txt')])
    
    return Fname

In [13]:
#List of Labels - 0 for normal, 1 for murmur
labels = [0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,1]
len(labels)

23

In [14]:
#They were originally named differently and I have renamed and numbered them for ease of access.
#The michigan heart sound recordings are in mp3 format
#The recordings are converted to wav format for easy processing as most python libraries require the audio format to be wav.

def converttowav(filepath):
  destinationpath = filepath.split('.')[0] + '.wav'
  sound = AudioSegment.from_mp3(filepath)
  sound.export(destinationpath, format = "wav")

filepathslist = list()
for i in range(23):
  filepath = str(i+1) + '.mp3'
  converttowav(filepath)
  filepathslist.append(str(i+1)+'.wav')

id = list()
fl = list()
for i in range(23):
  id.append(str(i+1))
  fl.append(0)

In [15]:
#nlabels = labels
#for i in range(77):
#  x = random.randint(1,23)
#  id.append(str(i+1))
#  filepathslist.append(str(x)+'.wav')
#  fl.append(0)
#  nlabels.append(labels[x-1])

In [16]:
#Creating an empty dataframe with columns to store filenames and the respective labels
dataframe = pd.DataFrame(columns = ['new ID', 'wav filename','flag value', 'labels'])

In [17]:
filepathslist

['1.wav',
 '2.wav',
 '3.wav',
 '4.wav',
 '5.wav',
 '6.wav',
 '7.wav',
 '8.wav',
 '9.wav',
 '10.wav',
 '11.wav',
 '12.wav',
 '13.wav',
 '14.wav',
 '15.wav',
 '16.wav',
 '17.wav',
 '18.wav',
 '19.wav',
 '20.wav',
 '21.wav',
 '22.wav',
 '23.wav']

In [18]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [19]:
labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [20]:
#Inserting filename and label values into dataframe
dataframe['new ID'] = id
dataframe['wav filename'] = filepathslist
dataframe['flag value'] = fl
dataframe['labels'] = labels

In [82]:
dataframenew = pd.DataFrame(columns = ['new ID', 'wav filename', 'flag value', 'labels'])

In [22]:
import random

In [23]:
#Get only murmur recordings
def getrecmur(inputdir):
    fname = getfilenames(inputdir)
    items = []
    rinfo = []
    recf = []
    labels = []
    recfile = []
    for i in fname:
        i = inputdir + '/' + i
        with open(i, 'r') as f:
            content = f.readlines()
        numloc = int(content[0].split(' ')[1])
        for i in range(numloc):
            for p in range(len(content)):
                t = content[p].split('\n')
                for it in t:
                    if it.startswith('#Murmur:'):
                        tar = it.split(': ')[1]
                        if tar == 'Present':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 1
                          labels.append(l)
                        elif tar == 'Absent':
                          recinfo = content[i+1].split(' ')
                          rinfo.append(recinfo)
                          l = 0
                          labels.append(l)
    for items in rinfo:
        for i in range(len(items)):
            if items[i].endswith('.wav'):
                recfile = input_dir + '/' + items[i]
        recf.append(recfile)
    return rinfo, recf, labels

rin, ref, label = getrecmur(input_dir)

In [24]:
len(rin)

3007

In [79]:
newID = list()
wavfilename = list()
newlabels = list()
flagvalue = list()
for i in range(100):
  x = random.randint(1,23)
  newID.append(str(i+1))
  wavfilename.append(str(x)+'.wav')
  flagvalue.append(0)
  newlabels.append(labels[x-1])

x = 100
for i in range(900):
  x = x + 1
  wavfilename.append(ref[i])
  newlabels.append(label[i])
  flagvalue.append(1)
  newID.append(str(x))

In [80]:
len(newID)

1000

In [83]:
dataframenew['new ID'] = newID
dataframenew['wav filename'] = wavfilename
dataframenew['flag value'] = flagvalue
dataframenew['labels'] = newlabels
dataframenew

,new ID,wav filename,flag value,labels
0,1,23.wav,0,1
1,2,6.wav,0,1
2,3,20.wav,0,0
3,4,21.wav,0,1
4,5,6.wav,0,1
...,...,...,...,...
995,996,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
996,997,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
997,998,/content/drive/MyDrive/ReCirCor/training_data/...,1,0
998,999,/content/drive/MyDrive/ReCirCor/training_data/...,1,0


In [28]:
dataframe

,new ID,wav filename,flag value,labels
0,1,1.wav,0,0
1,2,2.wav,0,0
2,3,3.wav,0,0
3,4,4.wav,0,0
4,5,5.wav,0,0
5,6,6.wav,0,1
6,7,7.wav,0,1
7,8,8.wav,0,1
8,9,9.wav,0,1
9,10,10.wav,0,1


In [84]:
dataframenew['labels'].value_counts()

0    716
1    284
Name: labels, dtype: int64

In [30]:
X_train = dataframe['new ID']
y_Train = dataframe['labels']

In [85]:
X_test = dataframenew['new ID']
y_Test = dataframenew['labels']

In [32]:
#X_train, X_test, y_Train, y_Test = train_test_split(dataframenew['new ID'], dataframenew['labels'], train_size=0.8, shuffle=True)

In [33]:
X_train

0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
16    17
17    18
18    19
19    20
20    21
21    22
22    23
Name: new ID, dtype: object

In [86]:
X_test

0         1
1         2
2         3
3         4
4         5
       ... 
995     996
996     997
997     998
998     999
999    1000
Name: new ID, Length: 1000, dtype: object

In [35]:
#Computing Features - 

#Discrete Wavelet Transform
def computeDWTs(x):
    #approximation and detail coefficients
    res = np.array([])
    ca, cd = pywt.dwt(x, 'db8')
    #Compute mean, stdev, skew, kurtosis
    camean = np.mean(ca)
    res = np.hstack((res,camean))
    castdev = tstd(ca)
    res = np.hstack((res,castdev))
    caskew = skew(ca)
    res = np.hstack((res,caskew))
    cakurtosis = kurtosis(ca)
    res = np.hstack((res,cakurtosis))
    cdmean = np.mean(cd)
    res = np.hstack((res,cdmean))
    cdstdev = tstd(cd)
    res = np.hstack((res,cdstdev))
    cdskew = skew(cd)
    res = np.hstack((res,cdskew))
    cdkurtosis = kurtosis(cd)
    res = np.hstack((res, cdkurtosis))
    return res

#Computing the Spectral Entropy of the signal - Shannon Entropy of the Power Spectral Density (PSD) of the data
def SpectralEntropy(Signal, SampleRate, method = 'fft', nperseg = None, normalize = False):
    Signal = np.array(Signal)
    if method == 'fft':
      _, psd = periodogram(Signal, SampleRate)
    elif method == 'welch':
      _, psd = welch(Signal, SampleRate, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

#Computing the Power Spectral Density(PSD) of signal - 
def computepsd(x):
    freq, psd = signal.welch(x)
    return psd

#Computing Mel-frequency cepstral coefficients, Gammatone frequency cepstral coefficients, RMS Energy, Zero-crossing rate, mean, variance, skew, and kurtosis of the signal
def computeMFCC(x, sr):
    mfccs = np.mean(librosa.feature.mfcc(y = x, sr = sr, n_mfcc = 13).T, axis = 0)
    return mfccs

def computeGFCC(file_name):
    gfcc = get_features(file=file_name, feature_names=["gfcc"])
    gfcc = np.array(gfcc['audio'][file_name]['features'])
    return gfcc

def computeRMSEn(x):
    rmsen = np.mean(librosa.feature.rms(x, frame_length=2048, hop_length=1024, center=True).T, axis = 0)
    return rmsen

def computeZCR(x):
    zcr = np.mean(librosa.feature.zero_crossing_rate(y = x, frame_length=2048, hop_length=1024).T, axis = 0)
    return zcr

def computestats(x):
#Mean, Variance, Skew, Kurtosis
    stats = []
    me = np.mean(x)
    stats.append(me)
    variance = np.var(x)
    stats.append(variance)
    sk = skew(x)
    stats.append(sk)
    kurt = kurtosis(x)
    stats.append(kurt)
    return stats

In [36]:
#Function to extract the required features
def extractfeatures(file_name, x, sr, *, mfcc: bool = False, dwt: bool = False, spectral_entropy: bool = False, psd: bool = False, gfcc: bool = False, rmse: bool = False, zcr: bool = False, stats: bool = False):
    finalfeatures = np.array([])
    if mfcc:
        mfccs = computeMFCC(x, sr)
        
        finalfeatures = np.hstack((finalfeatures, mfccs))
    if dwt:
        dwtr = computeDWTs(x)
        finalfeatures = np.hstack((finalfeatures, dwtr))
    if spectral_entropy:
        specen = SpectralEntropy(x, sr)
        finalfeatures = np.hstack((finalfeatures,specen))
    if psd:
        psds = computepsd(x)
        finalfeatures = np.hstack((finalfeatures, psds))
    if gfcc:
        gfccs = computeGFCC(file_name)
        finalfeatures = np.hstack((finalfeatures, gfccs))
    if rmse:
        rmses = computeRMSEn(x)
        finalfeatures = np.hstack((finalfeatures, rmses))
    if zcr:
        zcrs = computeZCR(x)
        finalfeatures = np.hstack((finalfeatures, zcrs))
    if stats:
        stats1 = computestats(x)
        finalfeatures = np.hstack((finalfeatures, stats1))
    return finalfeatures

In [37]:
#loads audio and extracts required features
def load_file_data(file_num, duration=15, sr = 4000):
    input_length=sr*duration
    notfeatures = []
    i = 0
    for file_no in file_num:
        try:
            #sound_file = folder + '/' + file_name
            sfile = int(file_no)
            sound_file = dataframenew['wav filename'][sfile-1]
            print ("load file ", i, sound_file)
            x, sr = librosa.load(sound_file,duration=duration,res_type='kaiser_fast')
            dur = librosa.get_duration(y=x, sr=sr)
            #Padding audio signal
            if (round(dur) < duration):
                print ("fixing audio length:", file_no, sound_file)
                y = librosa.util.fix_length(x, input_length)
            feature = extractfeatures(sound_file, x, sr, mfcc = True, dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True)
            #dwt = True, psd = True, gfcc = True, rmse = True, zcr = True, stats = True
            notfeatures.append(feature)          
        except Exception as e:
            print(e)
            #notfeatures = np.asarray(features, dtype=np.float32)
        i = i+1 
    return notfeatures

In [38]:
x_train = []
x_train = load_file_data(X_train)

load file  0 8.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 8.wav
load file  1 8.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 8.wav
load file  2 3.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 3.wav
load file  3 19.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 19.wav
load file  4 17.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 17.wav
load file  5 12.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 12.wav
load file  6 22.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 22.wav
load file  7 2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 2.wav
load file  8 5.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : 5.wav
load file  

In [87]:
x_test = []
x_test = load_file_data(X_test)

Streaming output truncated to the last 5000 lines.
        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_AV_1.wav
load file  289 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_AV_2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_AV_2.wav
load file  290 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_1.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_1.wav
load file  291 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_2.wav

        
 Extracting features gfcc 

        
Computing features for file 1 of 1 : /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_PV_2.wav
load file  292 /content/drive/MyDrive/ReCirCor/training_data/training_data/49748_

In [39]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

In [40]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [41]:
x_test.shape

(250, 200)

In [42]:
x_train.shape

(23, 200)

In [43]:
y_Train.dtype

dtype('int64')

In [44]:
y_Train = np.asarray(y_Train).astype(np.float32)

In [45]:
y_Test = np.asarray(y_Test).astype(np.float32)

In [46]:
traindataframe = pd.DataFrame(columns = ['x_train ID', 'labels'])
testdataframe = pd.DataFrame(columns = ['x_test ID', 'labels'])
traindataframe['x_train ID'] = X_train
testdataframe['x_test ID'] = X_test
traindataframe['labels'] = y_Train
testdataframe['labels'] = y_Test

In [47]:
testdataframe

,x_test ID,labels
0,1,1.0
1,2,1.0
2,3,0.0
3,4,0.0
4,5,1.0
...,...,...
245,246,0.0
246,247,0.0
247,248,0.0
248,249,0.0


In [48]:
xtrainwavfiles = list()
trainflagval = list()
for i in X_train:
  num = int(i)
  xtrainwavfiles.append(dataframe['wav filename'][num-1])
  trainflagval.append(dataframe['flag value'][num-1])

In [49]:
xtestwavfiles = list()
flagval = list()
for i in X_test:
  num = int(i)
  xtestwavfiles.append(dataframenew['wav filename'][num-1])
  flagval.append(dataframenew['flag value'][num-1])

In [50]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [52]:
pca = PCA(n_components=20)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.fit_transform(x_test)

In [53]:
x_train_pca.shape

(23, 20)

In [54]:
x_test_pca.shape

(250, 20)

In [55]:
dictofpca = defaultdict(list)

def getpcalist(x):
  for i in range(20):
    item = x[i]
    dictofpca[f"pca_{i}"].append(item)
  return dictofpca

for i in x_train_pca:
  pcanum = getpcalist(i)

testpcadict = defaultdict(list)

def getpcalist2(x):
  for i in range(20):
    item = x[i]
    testpcadict[f"pca_{i}"].append(item)
  return testpcadict

for i in x_test_pca:
  tpca = getpcalist2(i)

In [56]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train_pca, y_Train)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [57]:
clf.score(x_test_pca, y_Test)

0.468

In [58]:
y_pred1 = clf.predict(x_test_pca)
print("Classification Report : \n\n", classification_report(y_Test, y_pred1))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.57      0.03      0.06       134
         1.0       0.47      0.97      0.63       116

    accuracy                           0.47       250
   macro avg       0.52      0.50      0.34       250
weighted avg       0.52      0.47      0.32       250



In [59]:
df1 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value':flagval, 'Real Values':y_Test, 'Predicted Values':y_pred1})
df1

,Input,Wav filename,flag value,Real Values,Predicted Values
0,1,8.wav,0,1.0,1.0
1,2,8.wav,0,1.0,1.0
2,3,3.wav,0,0.0,1.0
3,4,19.wav,0,0.0,1.0
4,5,17.wav,0,1.0,1.0
...,...,...,...,...,...
245,246,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
246,247,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
247,248,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0
248,249,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,1.0


In [60]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [61]:
clf2 = LinearSVC(penalty='l2', loss='hinge')
clf2.fit(x_train_pca, y_Train)
clf2.score(x_test_pca, y_Test)

0.516

In [62]:
y_pred2 = clf2.predict(x_test_pca)
print("Classification Report : \n\n", classification_report(y_Test, y_pred2))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.53      0.84      0.65       134
         1.0       0.44      0.15      0.22       116

    accuracy                           0.52       250
   macro avg       0.48      0.49      0.43       250
weighted avg       0.49      0.52      0.45       250



In [63]:
df2 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval, 'Real Values':y_Test, 'Predicted Values':y_pred2})
df2

,Input,Wav filename,flag value,Real Values,Predicted Values
0,1,8.wav,0,1.0,0.0
1,2,8.wav,0,1.0,0.0
2,3,3.wav,0,0.0,0.0
3,4,19.wav,0,0.0,0.0
4,5,17.wav,0,1.0,0.0
...,...,...,...,...,...
245,246,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
246,247,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
247,248,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
248,249,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0


In [64]:
clf4 = SGDClassifier(loss = 'hinge').fit(x_train_pca, y_Train)

In [65]:
clf4.score(x_test_pca, y_Test)

0.524

In [66]:
y_pred4 = clf4.predict(x_test_pca)
#y_pred=np.argmax(y_pred, axis=1)
print("Classification Report : \n\n", classification_report(y_Test, y_pred4))

Classification Report : 

               precision    recall  f1-score   support

         0.0       0.54      0.84      0.65       134
         1.0       0.46      0.16      0.24       116

    accuracy                           0.52       250
   macro avg       0.50      0.50      0.45       250
weighted avg       0.50      0.52      0.46       250



In [67]:
df4 = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval, 'Real Values':y_Test, 'Predicted Values':y_pred4})
df4

,Input,Wav filename,flag value,Real Values,Predicted Values
0,1,8.wav,0,1.0,0.0
1,2,8.wav,0,1.0,0.0
2,3,3.wav,0,0.0,0.0
3,4,19.wav,0,0.0,0.0
4,5,17.wav,0,1.0,0.0
...,...,...,...,...,...
245,246,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
246,247,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
247,248,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0
248,249,/content/drive/MyDrive/ReCirCor/training_data/...,1,0.0,0.0


In [68]:
svmwpca = SVC(C = 0.1, gamma = 0.01)
svmwpca.fit(x_train_pca, y_Train)
svmwpca.score(x_test_pca, y_Test)

0.464

In [69]:
y_pred = svmwpca.predict(x_test_pca)
print(classification_report(y_Test, y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       134
         1.0       0.46      1.00      0.63       116

    accuracy                           0.46       250
   macro avg       0.23      0.50      0.32       250
weighted avg       0.22      0.46      0.29       250



In [70]:
pcadataframe = pd.DataFrame({'Input':X_train, 'Wav filename':xtrainwavfiles, 'flag value': trainflagval})
#'Labels':y_Train

In [71]:
for i in pcanum:
  pcadataframe[i] = pcanum[i]

In [72]:
y_newt = []
for i in y_Train:
  y_newt.append(int(i))

In [73]:
pcadataframe['Labels'] = y_newt

In [74]:
pcadataframe

,Input,Wav filename,flag value,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,...,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,Labels
0,1,1.wav,0,-1.551936,1.938305,-0.255642,0.316070,0.451394,0.087917,0.284691,...,-0.084591,0.060459,0.045915,2.534965e-07,-1.002962e-07,3.648065e-08,1.334789e-08,1.883682e-08,3.037590e-09,0
1,2,2.wav,0,-1.551935,1.938304,-0.255641,0.316070,0.451394,0.087917,0.284691,...,-0.084591,0.060459,0.045915,2.932030e-07,1.055872e-07,-2.852412e-08,-7.283053e-09,-1.535625e-08,1.031688e-09,0
2,3,3.wav,0,-0.626084,-1.233526,-0.014135,-0.558470,-0.526756,-0.147254,0.158348,...,-0.092671,0.105836,-0.046932,2.932218e-07,8.736838e-08,-5.222091e-08,4.999315e-09,-3.047266e-08,-1.552751e-09,0
3,4,4.wav,0,-0.544726,-1.851640,-0.692098,0.480921,1.353769,-0.318927,-0.612494,...,0.255160,-0.165361,-0.024032,2.738861e-07,-1.010261e-09,5.408190e-10,3.682456e-10,1.061601e-10,3.177344e-12,0
4,5,5.wav,0,2.617121,0.901453,1.819623,-0.204156,0.345612,-0.569572,0.293989,...,-0.012248,-0.078366,-0.004235,2.638344e-07,3.005804e-08,2.287569e-08,9.715789e-08,2.021905e-09,2.903252e-10,0
5,6,6.wav,0,2.565320,0.481694,-1.238580,0.042170,-0.431788,0.233054,-0.075341,...,0.002811,-0.021647,0.021581,2.724710e-07,5.926452e-08,9.628960e-08,-2.447196e-08,-1.834626e-08,-1.113527e-09,1
6,7,7.wav,0,0.330268,-1.746993,0.072974,-1.302296,1.796973,0.327810,-0.216942,...,-0.257621,0.108864,-0.011228,2.738856e-07,-1.010262e-09,5.408184e-10,3.682566e-10,1.061594e-10,3.177975e-12,1
7,8,8.wav,0,-1.345411,-0.801719,0.008349,0.043577,-0.005009,0.191099,0.937738,...,0.303010,-0.166389,-0.106628,2.738861e-07,-1.010221e-09,5.408147e-10,3.682313e-10,1.061569e-10,3.176445e-12,1
8,9,9.wav,0,-0.926320,-1.547507,-0.649790,-0.170231,-0.207830,-0.173449,0.333207,...,0.011729,-0.254699,0.299312,2.738861e-07,-1.010260e-09,5.408337e-10,3.682494e-10,1.061718e-10,3.181198e-12,1
9,10,10.wav,0,-0.626084,-1.233525,-0.014135,-0.558469,-0.526755,-0.147254,0.158348,...,-0.092671,0.105836,-0.046932,2.700616e-07,-1.056948e-08,1.806224e-08,-7.006732e-09,1.132123e-08,2.038123e-08,1


In [75]:
testpcadict = defaultdict(list)
def getpcalist(x):
  for i in range(20):
    item = x[i]
    testpcadict[f"pca_{i}"].append(item)
  return testpcadict

for i in x_test_pca:
  tpca = getpcalist(i)

In [76]:
pcaresult = pd.DataFrame({'Input':X_test, 'Wav filename':xtestwavfiles, 'flag value': flagval})

for i in tpca:
  pcaresult[i] = tpca[i]

y_newt2 = []
for i in y_Test:
  y_newt2.append(int(i))

pcaresult['Real Values'] = y_newt2
pcaresult['Predicted Values'] = y_pred2

In [77]:
pcaresult

,Input,Wav filename,flag value,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,...,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,Real Values,Predicted Values
0,1,8.wav,0,-2666.698730,-143.609268,-53.142937,8.267267,-9.391267,-4.945264,0.431377,...,1.988924,-0.248567,0.390781,0.347281,-0.031535,-0.553095,0.076376,-0.045691,1,0.0
1,2,8.wav,0,-2666.710449,-143.604584,-53.144276,8.267334,-9.389450,-4.947383,0.430920,...,1.989396,-0.245787,0.391084,0.347146,-0.027914,-0.551035,0.082381,-0.045065,1,0.0
2,3,3.wav,0,-2666.895264,-143.726181,-53.294106,8.182108,-9.479318,-5.410223,0.950188,...,-0.202833,-0.140085,-0.480582,-0.256235,0.681402,0.670444,0.179134,-0.036394,0,0.0
3,4,19.wav,0,-2666.836914,-143.715393,-53.290630,8.189341,-9.487407,-5.636826,0.752663,...,-0.090745,-1.078009,-0.390357,-0.026751,0.480624,-0.893120,0.366255,0.073657,0,0.0
4,5,17.wav,0,-2666.235352,-143.248276,-52.719090,8.429333,-9.266914,-4.426803,0.892550,...,0.736228,2.199312,-0.286397,-0.372598,-1.215960,-0.129563,0.030368,0.231563,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,246,/content/drive/MyDrive/ReCirCor/training_data/...,1,-2271.533203,-10.012648,-53.045773,11.659536,-9.317817,17.222595,-6.441645,...,0.378147,-0.343108,0.352674,-0.375362,0.000759,0.329890,-0.134680,-0.817778,0,0.0
246,247,/content/drive/MyDrive/ReCirCor/training_data/...,1,-2565.474609,-97.776093,-43.408733,2.925631,-5.501373,5.382388,-0.105425,...,-0.264000,0.832112,0.780868,-1.111161,0.128031,-0.282179,-0.150408,-0.289001,0,0.0
247,248,/content/drive/MyDrive/ReCirCor/training_data/...,1,-2587.418701,-107.727715,-44.470840,6.694047,-7.535556,0.966458,-7.081316,...,0.729054,-0.940676,0.833513,-0.863150,-0.482123,0.011873,-0.265841,-0.489287,0,0.0
248,249,/content/drive/MyDrive/ReCirCor/training_data/...,1,-2130.488281,-10.030440,-40.900204,3.331004,-5.519012,2.202714,-2.828770,...,0.791241,-0.896038,0.816907,-0.359743,-0.146928,0.038469,-0.106102,-0.143606,0,0.0


In [88]:
pcadataframe.to_csv('23ctrainpcalinearsvc-1.csv')
pcaresult.to_csv('23to250testcnpcalinearsvc-1.csv')